In [30]:
import numpy as np
import pandas as pd
import array_to_latex as a2l

In [35]:
eps = 0.001
alpha, beta = 0.1*13, 0.1*13
N = 4
A = np.array([
    [10 + alpha, -1, 0.2, 2],
    [1, 12-alpha, -2, 0.1],
    [0.3, -4, 12-alpha, 1],
    [0.2, -0.3, -0.5, 8-alpha]
])
Adiag = np.diagonal(A)
B = np.array([1+beta, 2-beta, 3, 1])

F = np.copy(A)
for i in range(N):
    F[i,i] = 0
for i in range(N):
    for j in range(N):
        F[i,j] = F[i,j]/Adiag[i]
Fup = np.copy(F)
Fdown = np.copy(F)
for i in range(N):
    for j in range(N):
        if j > i:
            Fdown[i,j] = 0
        else:
            Fup[i,j] = 0
C = np.copy(B)
for i in range(N):
    C[i] = C[i]/Adiag[i]

a2l.to_ltx(A, frmt = '{:6.2f}', arraytype = 'matrix')

\begin{matrix}
   11.30 &  -1.00 &    0.20 &    2.00\\
    1.00 &   10.70 &  -2.00 &    0.10\\
    0.30 &  -4.00 &   10.70 &    1.00\\
    0.20 &  -0.30 &  -0.50 &    6.70
\end{matrix}


In [9]:
def matrixMultByVector(A, x):
    return np.array([ 
        sum([ A[i,j]*x[j] for j in range(N) ]) for i in range(N)
    ])

In [10]:
def simpleNextIter(x):
    return C - matrixMultByVector(F, x)

def zeidelNextIter(x):
    res = np.zeros(N)
    for i in range(N):
        res[i] = C[i] - matrixMultByVector(Fdown, res)[i] - matrixMultByVector(Fup, x)[i] 
    return res

def norm(x):
    return np.max(np.abs(x))

def countDelta(xOld, xNew):
    deltaBig = findFNorm()/(1-findFNorm())*norm(xOld-xNew)
    deltaSmall = deltaBig/norm(xNew)
    return deltaBig, deltaSmall

def checkStop(xOld, xNew):
    return norm(xOld-xNew) < eps
        
def findFNorm():
    return max( np.sum(np.abs(F[i])) for i in range(N) )

In [73]:
def run(method):
    nextIter = simpleNextIter if method == 'simple' else zeidelNextIter
    df = pd.DataFrame({
        'DELTA': [],
        'delta': [],
    })
    xOld = C
    xNew = nextIter(xOld)
    i = 0
    DELTA, delta = countDelta(xOld = xOld, xNew = xNew)
    app = { 
        'delta': delta,
        'DELTA': DELTA
    }
    df = df.append(app, ignore_index=True)
    
    while not checkStop(xOld,xNew):
        i += 1
        xNew, xOld = nextIter(xNew), xNew
        DELTA, delta = countDelta(xOld = xOld, xNew = xNew)
        app = { 
            'delta': delta,
            'DELTA': DELTA
        }
        df = df.append(app, ignore_index=True)
        
    print(df)
    df.to_csv("simple.scv")
    return xNew, delta

def showMethod(method):
    xNew, delta = run(method)
    check = matrixMultByVector(A, xNew) 
    print()
    print("CHECK:", check)
    print("WANT TO GET", B)
    a2l.to_ltx(xNew, frmt = '{:6.8f}', arraytype = 'matrix')
    a2l.to_ltx(abs(check-B), frmt = '{:6.10}', arraytype = 'matrix')
    print("DIFFERENCE", check - B)
    print("NORM F", findFNorm())

In [74]:
showMethod('simple')

      DELTA     delta
0  0.031397  0.110096
1  0.010811  0.036499
2  0.002034  0.006845
3  0.000681  0.002287

CHECK: [2.30028559 0.69862361 2.99936744 0.99959147]
WANT TO GET [2.3 0.7 3.  1. ]
\begin{matrix}
  0.17717356 &  0.10280586 &  0.29782299 &  0.17073284
\end{matrix}
\begin{matrix}
  0.0002855925357 &  0.001376385834 &  0.0006325647992 &  0.0004085334096
\end{matrix}
DIFFERENCE [ 0.00028559 -0.00137639 -0.00063256 -0.00040853]
NORM F 0.4953271028037384


/tmp/ipykernel_217709/2663720278.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(app, ignore_index=True)
/tmp/ipykernel_217709/2663720278.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(app, ignore_index=True)
/tmp/ipykernel_217709/2663720278.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(app, ignore_index=True)
/tmp/ipykernel_217709/2663720278.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(app, ignore_index=True)


In [75]:
showMethod('zeidel')

      DELTA     delta
0  0.033744  0.112952
1  0.003272  0.010981
2  0.000280  0.000940

CHECK: [2.30010388 0.70016017 2.99997808 1.        ]
WANT TO GET [2.3 0.7 3.  1. ]
\begin{matrix}
  0.17715646 &  0.10297131 &  0.29793517 &  0.17081010
\end{matrix}
\begin{matrix}
  0.0001038826492 &  0.0001601705369 &  2.192025484\times 10^{-05} &     0.0
\end{matrix}
DIFFERENCE [ 1.03882649e-04  1.60170537e-04 -2.19202548e-05  0.00000000e+00]
NORM F 0.4953271028037384


/tmp/ipykernel_217709/2663720278.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(app, ignore_index=True)
/tmp/ipykernel_217709/2663720278.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(app, ignore_index=True)
/tmp/ipykernel_217709/2663720278.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(app, ignore_index=True)
